In [ ]:
def absolute_error(data, targets):
    return np.mean(abs(data-targets))

In [28]:
import numpy as np
import math


class RBF_network:

    def __init__(self, input_dims, output_dims, num_hidden_nodes=5, learning_rate=0.001, min_val = 0, max_val=2*math.pi):
        self.hidden_nodes = num_hidden_nodes
        self.w = np.random.normal(0, 1, (num_hidden_nodes, input_dims))
        
        #position RBFs
        self.centers = np.linspace(min_val, max_val, self.hidden_nodes)
        self.sigma = 1/np.sqrt(2*self.hidden_nodes)
        self.lr = learning_rate

    def train_leastsquares(self, data, targets, epochs=100):
        old_w = np.full((self.w.shape), 100)
        phi = np.zeros((len(data), self.hidden_nodes))
        for epoch in range(epochs):
            # Forward pass
            for i in range(len(self.centers)):
                phi[:, i] = np.exp((data - self.centers[i]) / 2 * self.sigma ** 2)
            print(phi.shape)
            output = np.dot(phi, self.w)

            # Backward pass with least squares
            self.w = np.dot(np.linalg.pinv(phi), output) ##is this correct?
            diff = abs(old_w - self.w)
            if np.max(diff) < 10e-3:
                return output
            
            old_w = self.w
        return output

    def train_delta_rule_online(self, data, targets, epochs=100):
        phi = np.zeros((len(self.centers)))
        old_w = 100
        for epoch in range(epochs):
            for idx, x in enumerate(data):
            # forward
                self.phi = np.exp((x - self.centers) / (2 * self.sigma ** 2))
                print(phi.shape)
                output = np.dot(phi, self.w)

                # backward
                error = output - targets[idx]
                self.w -= self.lr * error * phi
                diff = abs(old_w - self.w)
                if np.max(diff) < 10e-3:
                    return output
                       
                old_w = self.w
        return output
    
    def train_delta_rule_batch(self, data, targets, epochs=100, competitive=False):
        old_w = np.full((self.w.shape), 100)
        phi = np.zeros((len(data), self.hidden_nodes))
        for epoch in range(epochs):
            # Forward pass
            for i in range(len(self.centers)):
                phi[:, i] = np.exp((data - self.centers[i]) / 2 * self.sigma ** 2)
            print(phi.shape)
            output = np.dot(phi, self.w)
            
            #backward pass with delta rule:
            error = output - targets
            self.w -= self.lr * error * phi
            if competitive:
                x = np.random.choice(data)
                distance = x-self.centers
                winner = np.argmin(distance)
                self.centers[winner] += self.lr*distance
            diff = abs(old_w - self.w)
                if np.max(diff) < 10e-3:
                    return output
                       
                old_w = self.w
        return output
          

    def predict_leastsquares(self, data):
        phi = np.zeros((len(data), self.hidden_nodes))
        for i in range(len(self.centers)):
            phi[:, i] = np.exp((data - self.centers[i]) / 2 * self.sigma ** 2)
            print(phi.shape)
            output = np.dot(phi, self.w)
        return output
    
    def predict_delta_rule_online(self, data):
        phi = np.zeros((len(self.centers), 1))
        for idx, x in enumerate(data):
            # forward
                self.phi = np.exp((x - self.centers) / (2 * self.sigma ** 2))
                print(phi.shape)
                output = np.dot(phi, self.w)
        return output
    
    def predict_delta_rule_batch(self, data):
        phi = np.zeros((len(data), self.hidden_nodes))
        for i in range(len(self.centers)):
            phi[:, i] = np.exp((data - self.centers[i]) / 2 * self.sigma ** 2)
            print(phi.shape)
            output = np.dot(phi, self.w)
        return output
        


In [ ]:
def test_delta(trainData, trainTargets, testData, testTargets):
    train_errors = []
    test_errors = []
    mean_errors = []
    errors_std = []
    
    for i in range(1,50):
        for j in range(50):
            rbf = RBF_network(1,1,num_hidden_nodes = i)
            train_out = rbf.train_delta_rule_batch(trainData, trainTargets)
            test_out = rbf.predict_delta_rule_batch(testData)
            test_errors += [absolute_error(testData, testTargets)]
        mean_errors += [np.mean(test_errors)]
        errors_std += [np.std(test_errors)]
    return mean_errors, errors_std


In [ ]:
def test_least_squares(trainData, trainTargets, testData, testTargets):
    train_errors = []
    test_errors = []
    mean_errors = []
    errors_std = []
    
    for i in range(1,50):
        for j in range(50):
            rbf = RBF_network(1,1,num_hidden_nodes = i)
            train_out = rbf.train_leastsquares(trainData, trainTargets)
            test_out = rbf.predict_leastsquares(testData)
            test_errors += [absolute_error(testData, testTargets)]
        mean_errors += [np.mean(test_errors)]
        errors_std += [np.std(test_errors)]
    return mean_errors, errors_std

approximating sin function least squares batch method:

In [19]:


patterns = np.arange(0, 2*math.pi, 0.1)
targets_sin = np.array([math.sin(x) for x in patterns])
test_patterns = np.arange(0.05, 2*math.pi, 0.1)
test_targets = np.array([math.sin(x) for x in test_patterns])

rbf = RBF_network(1,1,5)
out = rbf.train_leastsquares(patterns, targets_sin)

mean_errors, errors_std = test_least_squares(patterns, targets_square, test_patterns, test_targets)

(63, 5)
(63, 5)


approximating square function least squares method:

In [ ]:
patterns = np.arange(0, 2*math.pi, 0.1)
targets_square = np.array([1 if(math.sin(x) >= 0) else -1 for x in patterns])
test_patterns = np.arange(0.05, 2*math.pi, 0.1)
test_targets = np.array([1 if(math.sin(x) >= 0) else -1 for x in test_patterns])

mean_errors, errors_std = test_least_squares(patterns, targets_square, test_patterns, test_targets)

approximating sin function delta batch method

In [ ]:
patterns = np.arange(0, 2*math.pi, 0.1)
targets_sin = np.array([math.sin(x) for x in patterns])
test_patterns = np.arange(0.05, 2*math.pi, 0.1)
test_targets = np.array([math.sin(x) for x in test_patterns])

mean_errors, errors_std = test_delta(patterns, targets_sin, test_patterns, test_targets)

approximating square function, delta batch method:

In [29]:

patterns = np.arange(0, 2*math.pi, 0.1)
targets_square = np.array([1 if(math.sin(x) >= 0) else -1 for x in patterns])

test_patterns = np.arange(0.05, 2*math.pi, 0.1)
test_targets = np.array([1 if(math.sin(x) >= 0) else -1 for x in test_patterns])

rbf = RBF_network(1,1,5)
out = rbf.train_delta_rule(patterns, targets_sin)

mean_errors, errors_std = test_delta(patterns, targets_sin, test_patterns, test_targets)

(5,)
(5,)


part 3.2 testing delta rule online learning with gaussian noise

In [ ]:
def test_delta_online(trainData, trainTargets, testData, testTargets):
    train_errors = []
    test_errors = []
    mean_errors = []
    errors_std = []
    
    for i in range(1,50):
        for j in range(50):
            rbf = RBF_network(1,1,num_hidden_nodes = i)
            train_out = rbf.train_delta_rule_online(trainData, trainTargets)
            test_out = rbf.predict_delta_rule_online(testData)
            test_errors += [absolute_error(testData, testTargets)]
        mean_errors += [np.mean(test_errors)]
        errors_std += [np.std(test_errors)]
    return mean_errors, errors_std

for sin function:

In [ ]:

patterns = np.arange(0, 2*math.pi, 0.1)
targets_sin = np.array([math.sin(x) for x in patterns])

noise = np.random.normal(0, 0.1, (patterns.shape))
patterns += noise 

test_patterns = np.arange(0.05, 2*math.pi, 0.1)
test_noise = np.random.normal(0, 0.1, (test_patterns.shape))

test_patterns += test_noise
test_targets = np.array([math.sin(x) for x in test_patterns])

mean_errors, errors_std = test_delta_online(patterns, targets_sin, test_patterns, test_targets)


for square function:

In [ ]:
patterns = np.arange(0, 2*math.pi, 0.1)
targets_square = np.array([1 if(math.sin(x) >= 0) else -1 for x in patterns])

noise = np.random.normal(0, 0.1, (patterns.shape))
patterns += noise

test_patterns = np.arange(0.05, 2*math.pi, 0.1)
test_noise = np.random.normal(0, 0.1, (test_patterns.shape))

test_patterns += test_noise
test_targets = np.array([1 if(math.sin(x) >= 0) else -1 for x in test_patterns])

mean_errors, errors_std = test_delta_online(patterns, targets_square, test_patterns, test_targets)